In [1]:
import pandas as pd
import numpy as np

from scipy import stats
from scipy.stats import uniform
from sympy import *
from scipy.stats import rv_continuous
import scipy

pd.options.display.float_format = '{:,.4f}'.format

import warnings
warnings.filterwarnings("ignore")

# Задача 1

Выявить зависимость между типом школ и уровнем интеллектуального развития 


<table>
  <caption></caption> 
  <tr>
    <th rowspan="2" class="first">Типы школ</th>
    <th colspan="3">Уровень интеллектуального развития</th>
  </tr>
  <tr>
    <td class="first">Низкий</td>
    <td class="first">Нормальный</td>
    <td class="first">Высокий</td>
  </tr>
  <tr>
    <td rowspan="1" class="first">Городская</td>
    <td>25</td>
    <td>50</td>
    <td>25</td>
  </tr>
    <td rowspan="1" class="first">Сельская</td>
    <td>52</td>
    <td>41</td>
    <td>7</td>
  <tr>
 </table>

In [2]:
df = pd.DataFrame([[25, 50, 25],
                   [52, 41, 7]], 
                  columns=['Низкий', 'Нормальный', 'Высокий'],
                  index=['Городская', 'Сельская'])

df

,Низкий,Нормальный,Высокий
Городская,25,50,25
Сельская,52,41,7


## Рассчет вручную

In [3]:
df['sum'] = df.sum(axis=1)

In [4]:
df = pd.concat([df, df.sum().to_frame().T])

In [5]:
df.index = ['Городская', 'Сельская', 'sum']

In [6]:
df

,Низкий,Нормальный,Высокий,sum
Городская,25,50,25,100
Сельская,52,41,7,100
sum,77,91,32,200


In [7]:
def get_teor_value(df):
    temp_list = []
    for i in range(df.shape[0] - 1):
        teor_val = (df.iloc[-1] * int(df.iloc[:,[-1]].loc[df.index[i]])) / int(df.iloc[:,[-1]].loc[df.index[-1]])
        temp_list.append(teor_val)
    return pd.DataFrame(temp_list, index=df.iloc[:-1].index)

In [8]:
teor_df = get_teor_value(df)

In [9]:
df = df.iloc[:-1].drop(df.columns[-1], axis=1).T

df

,Городская,Сельская
Низкий,25,52
Нормальный,50,41
Высокий,25,7


In [10]:
teor_df.index = ['Городская теор', 'Сельская теор']
teor_df = teor_df.drop(teor_df.columns[-1], axis=1).T
teor_df

,Городская теор,Сельская теор
Низкий,38.5000,38.5000
Нормальный,45.5000,45.5000
Высокий,16.0000,16.0000


In [11]:
merge_df = pd.concat([df, teor_df], axis=1)

merge_df

,Городская,Сельская,Городская теор,Сельская теор
Низкий,25,52,38.5000,38.5000
Нормальный,50,41,45.5000,45.5000
Высокий,25,7,16.0000,16.0000


In [12]:
merge_df['diff_город'] = (merge_df['Городская'] - merge_df['Городская теор']) ** 2 / merge_df['Городская теор']

merge_df['diff_село'] = (merge_df['Сельская'] - merge_df['Сельская теор']) ** 2 / merge_df['Сельская теор']

merge_df

,Городская,Сельская,Городская теор,Сельская теор,diff_город,diff_село
Низкий,25,52,38.5000,38.5000,4.7338,4.7338
Нормальный,50,41,45.5000,45.5000,0.4451,0.4451
Высокий,25,7,16.0000,16.0000,5.0625,5.0625


In [13]:
t = merge_df[['diff_город', 'diff_село']].sum().sum()
t

20.48264235764236

In [14]:
p = 1-stats.chi2.cdf(t, df=2)

p

3.5665697735942636e-05

## Рассчет пакетом stats

In [15]:
stats.chi2_contingency(df)

(20.48264235764236,
 3.566569773595299e-05,
 2,
 array([[38.5, 38.5],
        [45.5, 45.5],
        [16. , 16. ]]))

# Задача 2

Выполните рассчет критерия хи-квадрат Пирсона на уровне 5% нулевой гипотезезы о том, что пол и цвет глаз являются. неависимыми факторами на Криптоне. Теперь проведите соответствующий тест на уровне 5% нулевой гипотезы о том, что каждая из вероятностей ячейки равна 1/4. Прокомментируйте свой результат. 

<table>
  <caption></caption> 
  <tr>
    <th rowspan="2" class="first">Sex</th>
    <th colspan="2">Eye-clour</th>
  </tr>
  <tr>
    <td class="first">Blue</td>
    <td class="first">Brown</td>
  </tr>
  <tr>
    <td rowspan="1" class="first">Male</td>
    <td>19</td>
    <td>10</td>
  </tr>
    <td rowspan="1" class="first">Female</td>
    <td>9</td>
    <td>21</td>
  <tr>
 </table>

In [63]:
df = pd.DataFrame([[19, 10],
                   [9, 21]], 
                  columns=['Blue', 'Brown'],
                  index=['Male', 'Female'])

df

,Blue,Brown
Male,19,10
Female,9,21


#### Выполните рассчет критерия хи-квадрат Пирсона на уровне 5% нулевой гипотезезы о том, что пол и цвет глаз являются. неависимыми факторами на Криптоне. 

## Рассчет вручную

In [64]:
df['sum'] = df.sum(axis=1)
df = pd.concat([df, df.sum().to_frame().T])
df.index = ['Male', 'Female', 'sum']

df

,Blue,Brown,sum
Male,19,10,29
Female,9,21,30
sum,28,31,59


In [65]:
def get_teor_value(df):
    temp_list = []
    for i in range(df.shape[0] - 1):
        teor_val = (df.iloc[-1] * int(df.iloc[:,[-1]].loc[df.index[i]])) / int(df.iloc[:,[-1]].loc[df.index[-1]])
        temp_list.append(teor_val)
    return pd.DataFrame(temp_list, index=df.iloc[:-1].index)

In [66]:
teor_df = get_teor_value(df)
teor_df

,Blue,Brown,sum
Male,13.7627,15.2373,29.0000
Female,14.2373,15.7627,30.0000


In [67]:
df = df.iloc[:-1].drop(df.columns[-1], axis=1).T

df

,Male,Female
Blue,19,9
Brown,10,21


In [68]:
teor_df.index = ['Male_teor', 'Female_teor']
teor_df = teor_df.drop(teor_df.columns[-1], axis=1).T
teor_df

,Male_teor,Female_teor
Blue,13.7627,14.2373
Brown,15.2373,15.7627


In [69]:
merge_df = pd.concat([df, teor_df], axis=1)

merge_df

,Male,Female,Male_teor,Female_teor
Blue,19,9,13.7627,14.2373
Brown,10,21,15.2373,15.7627


In [70]:
merge_df['diff_male'] = (merge_df['Male'] - merge_df['Male_teor']) ** 2 / merge_df['Male_teor']

merge_df['diff_female'] = (merge_df['Female'] - merge_df['Female_teor']) ** 2 / merge_df['Female_teor']

merge_df

,Male,Female,Male_teor,Female_teor,diff_male,diff_female
Blue,19,9,13.7627,14.2373,1.9930,1.9266
Brown,10,21,15.2373,15.7627,1.8001,1.7401


In [71]:
t = merge_df[['diff_male', 'diff_female']].sum().sum()
t

7.459848244080723

In [72]:
p = stats.chi2.sf(t, df=1)

p

0.006309032516454838

## Рассчет пакетом stats

In [44]:
stats.chi2_contingency(df, correction=False)

(7.459848244080724,
 0.006309032516454837,
 1,
 array([[13.76271186, 14.23728814],
        [15.23728814, 15.76271186]]))

#### Теперь проведите соответствующий тест на уровне 5% нулевой гипотезы о том, что каждая из вероятностей ячейки равна 1/4.

In [56]:
new_df = df.copy(deep=True)

In [57]:
sum_df = new_df.sum().sum()
sum_df

59

In [58]:
new_df['Male_teor'] =  sum_df * 0.25
new_df['Female_teor'] = sum_df * 0.25

new_df

,Male,Female,Male_teor,Female_teor
Blue,19,9,14.7500,14.7500
Brown,10,21,14.7500,14.7500


In [59]:
new_df['diff_male'] = (new_df['Male'] - new_df['Male_teor']) ** 2 / new_df['Male_teor']

new_df['diff_female'] = (new_df['Female'] - new_df['Female_teor']) ** 2 / new_df['Female_teor']

new_df

,Male,Female,Male_teor,Female_teor,diff_male,diff_female
Blue,19,9,14.7500,14.7500,1.2246,2.2415
Brown,10,21,14.7500,14.7500,1.5297,2.6483


In [60]:
t = new_df[['diff_male', 'diff_female']].sum().sum()
t

7.6440677966101696

In [61]:
stats.chi2.sf(t, df=1)

0.00569593239138004

$p-value$ **меньше** уровня статистической значимости  $\alpha=0.05$ , принять нулевую гипотезe не можем. Следовательно, **каждая вероятность ячейки НЕ равна 1/4**. 